In [ ]:
from acr.utils import SOM_BLUE, ACR_BLUE, NNXR_GRAY

MAIN_EXP = 'swi'
SUBJECT_TYPE = 'som'
MAIN_COLOR = SOM_BLUE

In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
import xarray as xr
from scipy import stats
import pingouin as pg
from scipy.stats import shapiro, normaltest
# ---------------------------- EXTRAS --------------------------------#
from kdephys.plot.main import _title, bp_plot
import kdephys.utils.spectral as sp
bands = sp.bands

import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
from acr.utils import *
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pu')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'dag')

In [ ]:
subjects, exps = pu.get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
notebook_figure_root = f'{pu.PAPER_FIGURE_ROOT}/OFF_PERIODS'

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True)

In [ ]:
bindfs = [] 
oodfs = {}
for subject, exp in zip(subjects, exps):
        print(subject, exp)
        if subject in bindfs:
                oodf = dag.load_bindf_with_hybrid_format(subject, exp)
        else:
                oodf = dag.compute_hybrid_off_df(subject, exp, chan_threshold=12)
        oodf = acr.oo_utils.enhance_oodf(oodf, full_hyps[subject], hyp_dicts[subject])
        oodfs[subject] = oodf
full_oodf = pl.concat(oodfs.values())
off_means = full_oodf.group_by(['subject', 'probe', 'condition']).agg(pl.col('duration_rel').mean())

In [ ]:
hdf_freqs = []
freq_means = []
for subject in subjects:
    subdf = full_oodf.filter(pl.col('subject')==subject)
    subfreq = acr.oo_utils.calc_off_freq(subdf, window='5s')
    subfreq = acr.oo_utils.label_oodf_with_states(subfreq, full_hyps[subject])
    subfreq = acr.oo_utils.label_oodf_full_bl(subfreq)
    subfreq = acr.oo_utils.label_oodf_hyp_dict_conditions(subfreq, hyp_dicts[subject])    
    subfreq = acr.oo_utils.relativize_oodf(subfreq, ref_to_col='full_bl', ref_to_val='True', avg_method='mean', col_to_relativize='off_freq_per_s') 
    if subject == 'ACR_97':
        #slight issue w ACR_45 baseline NNXo, because of low FR detection in NREM sleep compared to wake in this mouse for some reason, will smooth out the baseline value using contra control probe
        subfreq = subfreq.with_columns(off_freq_per_s_bl=pl.lit(subfreq.prb('NNXr')['off_freq_per_s_bl'][0]))
        rel_values = subfreq['off_freq_per_s']/subfreq['off_freq_per_s_bl']
        subfreq = subfreq.with_columns(off_freq_per_s_rel=pl.lit(rel_values))
    hdf_freqs.append(subfreq)
freq = pl.concat(hdf_freqs)
freq = freq.filter(pl.col('condition')!='None')
freq_means = freq.group_by(['subject', 'probe', 'condition']).agg(pl.col('off_freq_per_s_rel').mean())

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(16, 6))
sns.boxplot(data=off_means, x='condition', y='duration_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, MAIN_COLOR], ax=ax[0], order=COND_ORDER)
sns.stripplot(data=off_means, x='condition', y='duration_rel', hue='probe', dodge=True, hue_order=['NNXr', 'NNXo'], palette=['red', 'black'], alpha=0.7, ax=ax[0], s=5, order=COND_ORDER)
#ax[0].set_ylim(0, 1.5)
ax[0].axhline(y=1, color='red', linestyle='--')
ax[0].set_title(f'Threshold: {12} Channels | OFF Period Duration Across all Subjects, Relative to Full Baseline')
sns.boxplot(data=freq_means, x='condition', y='off_freq_per_s_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, MAIN_COLOR], ax=ax[1], order=COND_ORDER)
sns.stripplot(data=freq_means, x='condition', y='off_freq_per_s_rel', hue='probe', dodge=True, hue_order=['NNXr', 'NNXo'], palette=['red', 'black'], alpha=0.7, ax=ax[1], s=5, order=COND_ORDER)
#ax[1].set_ylim(0, 1.5)
ax[1].axhline(y=1, color='red', linestyle='--')
ax[1].set_title(f'Threshold: {12} Channels | OFF Period Frequency Across all Subjects, Relative to Full Baseline')

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
nnxr = freq_means.cdn('stim').prb('NNXr').sort('subject')['off_freq_per_s_rel'].to_numpy()
nnxo = freq_means.cdn('stim').prb('NNXo').sort('subject')['off_freq_per_s_rel'].to_numpy()

fig_id = 'off-freq-during-STIM'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
acr.plots.gen_paired_boxplot(nnxr, nnxo, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR])
print(ax.get_ylim())
ax.set_yticks([0, 1, 2])
ax.set_ylim(-0.1, 2.05)
f.savefig(fig_path,dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
plt.rcdefaults()
acr.plots.supl()
nnxr = freq_means.cdn('stim').prb('NNXr').sort('subject')['off_freq_per_s_rel'].to_numpy()
nnxo = freq_means.cdn('stim').prb('NNXo').sort('subject')['off_freq_per_s_rel'].to_numpy()

nnxr_rel = nnxr/nnxr
nnxo_rel = nnxo/nnxr

fig_id = 'off-freq-during-STIM-NORMALIZED'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
acr.plots.gen_paired_boxplot(nnxr_rel, nnxo_rel, ax=ax, one_sided=True, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')
ax.get_ylim()

----------

In [ ]:
nnxr = off_means.cdn('stim').prb('NNXr').sort('subject')['duration_rel'].to_numpy()
nnxo = off_means.cdn('stim').prb('NNXo').sort('subject')['duration_rel'].to_numpy()

fig_id = 'off-duration-during-STIM'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
acr.plots.gen_paired_boxplot(nnxr, nnxo, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')
ax.get_ylim()

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
nnxr = off_means.cdn('stim').prb('NNXr').sort('subject')['duration_rel'].to_numpy()
nnxo = off_means.cdn('stim').prb('NNXo').sort('subject')['duration_rel'].to_numpy()

nnxr_rel = nnxr/nnxr
nnxo_rel = nnxo/nnxr

fig_id = 'off-duration-during-STIM-NORMALIZED'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
acr.plots.gen_paired_boxplot(nnxr_rel, nnxo_rel, ax=ax, one_sided=True, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')
ax.get_ylim()

--------------

In [ ]:
nnxr = off_means.cdn('rebound').prb('NNXr').sort('subject')['duration_rel'].to_numpy()
nnxo = off_means.cdn('rebound').prb('NNXo').sort('subject')['duration_rel'].to_numpy()

fig_id = 'off-duration-during-REBOUND'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
acr.plots.gen_paired_boxplot(nnxr, nnxo, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')
ax.get_ylim()

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
nnxr_rel = nnxr/nnxr
nnxo_rel = nnxo/nnxr

fig_id = 'off-duration-during-REBOUND-NORMALIZED'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
acr.plots.gen_paired_boxplot(nnxr_rel, nnxo_rel, ax=ax, one_sided=True, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')


-------------------

In [ ]:
nnxr = freq_means.cdn('rebound').prb('NNXr').sort('subject')['off_freq_per_s_rel'].to_numpy()
nnxo = freq_means.cdn('rebound').prb('NNXo').sort('subject')['off_freq_per_s_rel'].to_numpy()

fig_id = 'off-freq-during-REBOUND'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
acr.plots.gen_paired_boxplot(nnxr, nnxo, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
print(ax.get_ylim())
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')


In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
#stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
nnxr_rel = nnxr/nnxr
nnxo_rel = nnxo/nnxr

fig_id = 'off-freq-during-REBOUND-NORMALIZED'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
acr.plots.gen_paired_boxplot(nnxr_rel, nnxo_rel, ax=ax, one_sided=True, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, transparent=True, bbox_inches='tight')
